In [5]:
import oktoberfest as ok
%load_ext autoreload 
%autoreload 2
import pandas as pd 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


List of issues : 
1. Filter peptides function : is documented but I had to use filter peptide for model. max,min length and max charge was not clearly written what to write
2. Merge spectra function : would be nice if it could return a df instead of having to write the variable name.spectrum_data to see it. also spectrum_data bit was not documented so I could n't review what I created. 
3. annotate spectral library : 
- it is not documented at all in the api so one has to find in the source code. 
- is also confusing as it is annotating psms and in the input parameter I am supposed to input the "spectra library" ? which adds to the confusion. 
- It also does not return a value and that is also not documented. So if assigned to a value it return empty which creates problems down the line.
4. in prediction part you have to start with ce caliberation first before grpc which is also not documented

In [6]:
config = {
    "type": "Rescoring",
    "tag": "",
    "output": "./out",
    "inputs": {
        "search_results": "./msms.txt",
        "search_results_type": "Maxquant",
        "spectra": "./",
        "spectra_type": "raw"
    },
    "models": {
        "intensity": "Prosit_2020_intensity_HCD",
        "irt": "Prosit_2019_irt"
    },
    "prediction_server": "koina.proteomicsdb.org:443",
    "numThreads": 1,
    "fdr_estimation_method": "mokapot",
    "allFeatures": False,
    "regressionMethod":"spline",
    "ssl": False,
    "thermoExe": "ThermoRawFileParser.exe",
    "massTolerance": 20,
    "unitMassTolerance": "ppm"}

    Pre-Processing : 

Spectral preprocessing

In [11]:
output = '/home/karim/projects/oktoberfest/tutorials/out/'
therm = '/home/karim/non_git/essentials/therm/ThermoRawFileParser.exe'


Getting a list of raw files

In [12]:
list_of_spectra = ok.pp.list_spectra('/home/karim/projects/oktoberfest/tutorials/Oktoberfest_input','raw')
print(list_of_spectra)

[PosixPath('/home/karim/projects/oktoberfest/tutorials/Oktoberfest_input/GN20170722_SK_HLA_G0103_R1_02.raw'), PosixPath('/home/karim/projects/oktoberfest/tutorials/Oktoberfest_input/GN20170722_SK_HLA_G0103_R2_01.raw'), PosixPath('/home/karim/projects/oktoberfest/tutorials/Oktoberfest_input/GN20170722_SK_HLA_G0103_R2_02.raw'), PosixPath('/home/karim/projects/oktoberfest/tutorials/Oktoberfest_input/GN20170722_SK_HLA_G0103_R1_01.raw')]


Converting raw files to mzml and merging into one file

In [13]:
from pyteomics import mzml
import os

# Define the output directory where individual mzML files are stored
output_directory = '/home/karim/projects/oktoberfest/tutorials/out/' 

# Create a directory to store the mzML files if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Initialize an empty list to store the paths of the converted mzML files
conversion_results = []
# TODO you need to understand this code 
# Loop through the list of spectra files and convert each one
for file in list_of_spectra:
    # Generate a unique output file name for the mzML file
    base_filename = os.path.splitext(os.path.basename(file))[0]  # Remove file extension
    output_mzml_file = os.path.join(output_directory, f"{base_filename}.mzML")
    
    # Call the convert_spectra function with the current file and the output file path
    ok.pp.convert_spectra(file, output_file=output_mzml_file, thermo_exe=therm)
    
    # Append the path of the converted mzML file to the conversion_results list
    conversion_results.append(output_mzml_file)




2023-11-07 16:15:38,639 - INFO - spectrum_io.raw.thermo_raw::convert_raw_mzml Converting thermo rawfile to mzml with the command: mono /home/karim/non_git/essentials/therm/ThermoRawFileParser.exe --msLevel=2 -i /home/karim/projects/oktoberfest/tutorials/Oktoberfest_input/GN20170722_SK_HLA_G0103_R1_02.raw -b /home/karim/projects/oktoberfest/tutorials/out/GN20170722_SK_HLA_G0103_R1_02.mzML


2023-11-07 16:15:38 INFO Started parsing /home/karim/projects/oktoberfest/tutorials/Oktoberfest_input/GN20170722_SK_HLA_G0103_R1_02.raw
2023-11-07 16:15:43 INFO Processing 53354 MS scans
10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 

2023-11-07 16:17:14 INFO Finished parsing /home/karim/projects/oktoberfest/tutorials/Oktoberfest_input/GN20170722_SK_HLA_G0103_R1_02.raw
2023-11-07 16:17:14 INFO Processing completed 0 errors, 0 warnings
2023-11-07 16:17:15,030 - INFO - spectrum_io.raw.thermo_raw::convert_raw_mzml Converting thermo rawfile to mzml with the command: mono /home/karim/non_git/essentials/therm/ThermoRawFileParser.exe --msLevel=2 -i /home/karim/projects/oktoberfest/tutorials/Oktoberfest_input/GN20170722_SK_HLA_G0103_R2_01.raw -b /home/karim/projects/oktoberfest/tutorials/out/GN20170722_SK_HLA_G0103_R2_01.mzML
2023-11-07 16:17:15 INFO Started parsing /home/karim/projects/oktoberfest/tutorials/Oktoberfest_input/GN20170722_SK_HLA_G0103_R2_01.raw
2023-11-07 16:17:19 INFO Processing 519

Path of the mzml file

In [14]:
mzml = '/home/karim/projects/oktoberfest/tutorials/out/'

Loading the spectra from mzml file

In [15]:
spectra = ok.pp.load_spectra(mzml_file= mzml, parser= 'pyteomics')

2023-11-07 16:21:44,136 - INFO - spectrum_io.raw.msraw::read_mzml Reading mzML file: /home/karim/projects/oktoberfest/tutorials/out/GN20170722_SK_HLA_G0103_R1_01.mzML
2023-11-07 16:22:03,836 - INFO - spectrum_io.raw.msraw::read_mzml Reading mzML file: /home/karim/projects/oktoberfest/tutorials/out/GN20170722_SK_HLA_G0103_R2_02.mzML
2023-11-07 16:22:23,332 - INFO - spectrum_io.raw.msraw::read_mzml Reading mzML file: /home/karim/projects/oktoberfest/tutorials/out/GN20170722_SK_HLA_G0103_R2_01.mzML
2023-11-07 16:22:43,167 - INFO - spectrum_io.raw.msraw::read_mzml Reading mzML file: /home/karim/projects/oktoberfest/tutorials/out/GN20170722_SK_HLA_G0103_R1_02.mzML


In [16]:
spectra.head()

,RAW_FILE,SCAN_NUMBER,INTENSITIES,MZ,MZ_RANGE,RETENTION_TIME,MASS_ANALYZER,FRAGMENTATION
GN20170722_SK_HLA_G0103_R1_01_4,GN20170722_SK_HLA_G0103_R1_01,4,"[2115.24267578125, 4477.49169921875, 19077.226...","[101.2032699584961, 102.09208679199219, 123.11...",100.00-385.00,0.011957,FTMS,HCD
GN20170722_SK_HLA_G0103_R1_01_5,GN20170722_SK_HLA_G0103_R1_01,5,"[8681.994140625, 693.3976440429688, 580.188476...","[101.06001281738281, 105.36332702636719, 110.0...",100.00-395.00,0.014245,FTMS,HCD
GN20170722_SK_HLA_G0103_R1_01_6,GN20170722_SK_HLA_G0103_R1_01,6,"[5248.67822265625, 118141.4140625, 4605.001953...","[102.05547332763672, 102.09180450439453, 103.0...",100.00-325.00,0.016720,FTMS,HCD
GN20170722_SK_HLA_G0103_R1_01_7,GN20170722_SK_HLA_G0103_R1_01,7,"[523.4241333007812, 516.0610961914062, 18938.2...","[129.10244750976562, 141.6624298095703, 149.04...",100.00-410.00,0.019195,FTMS,HCD
GN20170722_SK_HLA_G0103_R1_01_8,GN20170722_SK_HLA_G0103_R1_01,8,"[503.8333435058594, 478.1268310546875, 592.651...","[101.03336334228516, 119.5694580078125, 132.63...",100.00-485.00,0.021689,FTMS,HCD


Peptide processing

Converting search results to Oktoberfest format.

In [17]:
peptide_out_file = '/home/karim/projects/oktoberfest/tutorials/out/peptide_output.tsv'
split_search = '/home/karim/projects/oktoberfest/tutorials/out/split_search'


In [19]:
ok.pp.convert_search(input_path= '/home/karim/projects/oktoberfest/tutorials/Oktoberfest_input/msms.txt',output_file = peptide_out_file , search_engine= 'maxquant')

2023-11-07 16:25:19,617 - INFO - spectrum_io.search_result.maxquant::read_result Reading msms.txt file


2023-11-07 16:25:20,013 - INFO - spectrum_io.search_result.maxquant::read_result Finished reading msms.txt file
2023-11-07 16:25:20,023 - INFO - spectrum_io.search_result.maxquant::update_columns_for_prosit Converting MaxQuant peptide sequence to internal format
2023-11-07 16:25:20,213 - INFO - spectrum_io.search_result.search_results::filter_valid_prosit_sequences #sequences before filtering for valid prosit sequences: 99672
2023-11-07 16:25:20,333 - INFO - spectrum_io.search_result.search_results::filter_valid_prosit_sequences #sequences after filtering for valid prosit sequences: 99202


In [20]:
peptide_csv = pd.read_csv('/home/karim/projects/oktoberfest/tutorials/out/peptide_output.tsv')

In [21]:
peptide_csv.head()

,RAW_FILE,SCAN_NUMBER,MODIFIED_SEQUENCE,PRECURSOR_CHARGE,SCAN_EVENT_NUMBER,MASS,SCORE,REVERSE,SEQUENCE,PEPTIDE_LENGTH
0,GN20170722_SK_HLA_G0103_R1_01,35161,AAAAAAYY,1,1,770.35990,8.0035,False,AAAAAAYY,8
1,GN20170722_SK_HLA_G0103_R1_02,51825,AAAAPPWAC[UNIMOD:4]FAAV,2,2,1301.62270,3.9695,True,AAAAPPWACFAAV,13
2,GN20170722_SK_HLA_G0103_R1_01,21329,AAAAVVSGPKRGRKKP,3,9,1591.96340,7.9138,True,AAAAVVSGPKRGRKKP,16
3,GN20170722_SK_HLA_G0103_R2_02,20501,AAAAVVSGPKRGRKKP,3,5,1591.96340,4.7582,True,AAAAVVSGPKRGRKKP,16
4,GN20170722_SK_HLA_G0103_R1_01,5650,AAAC[UNIMOD:4]RFVQ,3,8,921.44907,7.5676,True,AAACRFVQ,8


Loading search results

In [22]:
peptide_df = ok.pp.load_search(peptide_out_file)

In [23]:
peptide_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99202 entries, 0 to 99201
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   RAW_FILE           99202 non-null  object 
 1   SCAN_NUMBER        99202 non-null  int64  
 2   MODIFIED_SEQUENCE  99202 non-null  object 
 3   PRECURSOR_CHARGE   99202 non-null  int64  
 4   SCAN_EVENT_NUMBER  99202 non-null  int64  
 5   MASS               99202 non-null  float64
 6   SCORE              99202 non-null  float64
 7   REVERSE            99202 non-null  bool   
 8   SEQUENCE           99202 non-null  object 
 9   PEPTIDE_LENGTH     99202 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(3)
memory usage: 6.9+ MB


Filtering search results using given constraints.

In [24]:
filtered_peptides = ok.pp.filter_peptides_for_model(peptides = peptide_df , model='prosit')

In [25]:
filtered_peptides.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99202 entries, 0 to 99201
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   RAW_FILE           99202 non-null  object 
 1   SCAN_NUMBER        99202 non-null  int64  
 2   MODIFIED_SEQUENCE  99202 non-null  object 
 3   PRECURSOR_CHARGE   99202 non-null  int64  
 4   SCAN_EVENT_NUMBER  99202 non-null  int64  
 5   MASS               99202 non-null  float64
 6   SCORE              99202 non-null  float64
 7   REVERSE            99202 non-null  bool   
 8   SEQUENCE           99202 non-null  object 
 9   PEPTIDE_LENGTH     99202 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(3)
memory usage: 7.7+ MB


Generating Psms

In [26]:
psms = ok.pp.merge_spectra_and_peptides(spectra= spectra , search= filtered_peptides)

2023-11-07 16:25:26,253 - INFO - oktoberfest.preprocessing.preprocessing::merge_spectra_and_peptides Merging rawfile and search result
2023-11-07 16:25:26,420 - INFO - oktoberfest.preprocessing.preprocessing::merge_spectra_and_peptides There are 99202 matched identifications


In [27]:
psms.spectra_data

,RAW_FILE,SCAN_NUMBER,MODIFIED_SEQUENCE,PRECURSOR_CHARGE,SCAN_EVENT_NUMBER,MASS,SCORE,REVERSE,SEQUENCE,PEPTIDE_LENGTH,INTENSITIES,MZ,MZ_RANGE,RETENTION_TIME,MASS_ANALYZER,FRAGMENTATION
0,GN20170722_SK_HLA_G0103_R1_01,35161,AAAAAAYY,1,1,770.3599,8.0035,False,AAAAAAYY,8,"[324826.46875, 1255.1343994140625, 6190.715332...","[100.1125717163086, 100.1261215209961, 101.071...",100.00-805.00,78.086489,FTMS,HCD
1,GN20170722_SK_HLA_G0103_R1_02,51825,AAAAPPWAC[UNIMOD:4]FAAV,2,2,1301.6227,3.9695,True,AAAAPPWACFAAV,13,"[2698.22314453125, 3675.60693359375, 1123.8502...","[101.07110595703125, 102.05532836914062, 103.0...",100.00-685.00,116.195940,FTMS,HCD
2,GN20170722_SK_HLA_G0103_R1_01,21329,AAAAVVSGPKRGRKKP,3,9,1591.9634,7.9138,True,AAAAVVSGPKRGRKKP,16,"[1057.125732421875, 10181.4873046875, 33789.39...","[101.06056213378906, 102.05546569824219, 102.0...",100.00-1650.00,46.272255,FTMS,HCD
3,GN20170722_SK_HLA_G0103_R1_01,21329,PTALHKQRL,2,9,1062.6298,9.3575,False,PTALHKQRL,9,"[1057.125732421875, 10181.4873046875, 33789.39...","[101.06056213378906, 102.05546569824219, 102.0...",100.00-1650.00,46.272255,FTMS,HCD
4,GN20170722_SK_HLA_G0103_R2_02,20501,AAAAVVSGPKRGRKKP,3,5,1591.9634,4.7582,True,AAAAVVSGPKRGRKKP,16,"[870.1561279296875, 996.1369018554688, 39476.2...","[102.055419921875, 102.0848617553711, 102.0918...",100.00-1650.00,46.388269,FTMS,HCD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99197,GN20170722_SK_HLA_G0103_R2_01,35746,YYYKDGDYYEDEDEDEDE,3,4,2353.8183,1.6758,False,YYYKDGDYYEDEDEDEDE,18,"[1551.66748046875, 4759.90185546875, 5069.5161...","[101.07146453857422, 102.05550384521484, 103.0...",100.00-2425.00,82.511449,FTMS,HCD
99198,GN20170722_SK_HLA_G0103_R1_02,52630,YYYKDGDYYEDEDEDEDEDEENE,2,6,2970.0159,0.0000,False,YYYKDGDYYEDEDEDEDEDEENE,23,"[944.7178955078125, 737.7440185546875, 1518.81...","[101.0711669921875, 103.05423736572266, 110.07...",100.00-3050.00,118.207100,FTMS,HCD
99199,GN20170722_SK_HLA_G0103_R1_01,41703,YYYLDWANIKGDYE,2,12,1811.8043,6.2759,True,YYYLDWANIKGDYE,14,"[5452.5234375, 937.5213623046875, 9635.3115234...","[101.07122039794922, 101.1076889038086, 102.05...",100.00-945.00,93.073608,FTMS,HCD
99200,GN20170722_SK_HLA_G0103_R2_01,37870,YYYLDWANIKGDYE,2,12,1811.8043,6.2759,True,YYYLDWANIKGDYE,14,"[8445.8740234375, 1322.7825927734375, 15181.16...","[101.07130432128906, 102.049072265625, 102.055...",100.00-945.00,87.243202,FTMS,HCD


    Rescoring

Generating original features 

In [28]:
# adding the path for the original and rescored features.
original_features = '/home/karim/projects/oktoberfest/tutorials/out/original.tab'
rescore_features = '/home/karim/projects/oktoberfest/tutorials/out/rescore.tab'


Anotate the Psms

In [29]:
# No need to assign this function to a variable as it does not have a return.
ok.pp.annotate_spectral_library(psms= psms , mass_tol= 20 , unit_mass_tol= 'ppm' )

2023-11-07 16:25:29,012 - INFO - oktoberfest.preprocessing.preprocessing::annotate_spectral_library Annotating spectra...
2023-11-07 15:27:30,672 - INFO - spectrum_fundamentals.annotation.annotation::annotate_spectra Removed count    99202.000000
mean         0.000171
std          0.013090
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: removed_peaks, dtype: float64 redundant peaks
2023-11-07 16:27:30,693 - INFO - oktoberfest.preprocessing.preprocessing::annotate_spectral_library Finished annotating.


Generating features for the original

Predictions 
first perform ce caliberation and add to a variable as the function returns a pd series

In [30]:
alignment_library = ok.pr.ce_calibration(psms,url= config['prediction_server'] , intensity_model= config['models']['intensity'],  
                   irt_model= config['models']['irt'])

Inferring predictions for 32000 spectra with batch site 1000:   0%|          | 0/32 [00:00<?, ?it/s]

Inferring predictions for 32000 spectra with batch site 1000:   0%|          | 0/32 [00:00<?, ?it/s]

In [31]:
ce_alignment = alignment_library.spectra_data.groupby(by=["COLLISION_ENERGY"])["SPECTRAL_ANGLE"].mean()

In [32]:
best_ce = ce_alignment.idxmax()
psms.spectra_data['COLLISION_ENERGY']= best_ce

you have to understand this part to do it the caliberation. you need the collision energy and also the best collision energy for the file to be ready to predict the features of the rescored. Also apperently you need to split the psms file. 

In [33]:
gprc = ok.pr.grpc_predict(library = psms ,url= config['prediction_server'] , intensity_model= config['models']['intensity'],  
                   irt_model= config['models']['irt'], ssl= True , alignment= False)

Inferring predictions for 99202 spectra with batch site 1000:   0%|          | 0/100 [00:00<?, ?it/s]

Inferring predictions for 99202 spectra with batch site 1000:   0%|          | 0/100 [00:00<?, ?it/s]

In [34]:
psms.spectra_data['PREDICTED_IRT']

0         36.519104
1        148.557037
2        -29.577068
3        104.830978
4        -29.577068
            ...    
99197     46.457218
99198     47.603615
99199    115.418266
99200    115.418266
99201    115.418266
Name: PREDICTED_IRT, Length: 99202, dtype: float32

In [35]:
ok.re.generate_features( library= psms , search_type= 'rescore' , output_file= rescore_features , all_features=False , regression_method= 'spline' )

/home/karim/miniconda3/envs/oktoberfest/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


2023-11-07 15:30:07,933 - INFO - spectrum_fundamentals.metrics.percolator::get_indices_below_fdr Found 6601 (out of 58078) targets below 0.01             FDR using spectral_angle as feature
2023-11-07 15:30:07,934 - INFO - spectrum_fundamentals.metrics.percolator::apply_lda_and_get_indices_below_fdr Found 6601 targets and 41124 decoys as input for the LDA model
2023-11-07 15:30:08,441 - INFO - spectrum_fundamentals.metrics.percolator::get_indices_below_fdr Found 12500 (out of 58078) targets below 0.01             FDR using lda_scores as feature
2023-11-07 15:30:08,488 - INFO - spectrum_fundamentals.metrics.percolator::calc Median absolute error predicted vs observed retention time on targets < 1% FDR: 1.3319870130893658


In [36]:
ok.re.generate_features( library= psms , search_type= 'original' , output_file= original_features, all_features=False, regression_method= 'spline')

Merging the input 

Generating a list of Tab files 

In [37]:
rescore_percolator = '/home/karim/projects/oktoberfest/tutorials/out/percolator/'
original_percolator = '/home/karim/projects/oktoberfest/tutorials/out/percolator/'
plot = '/home/karim/projects/oktoberfest/tutorials/out/' 

In [38]:
#ok.re.merge_input(tab_list,combined_tab)

In [40]:
ok.re.rescore_with_percolator(rescore_features,rescore_percolator)

2023-11-07 16:32:16,731 - INFO - oktoberfest.rescore.rescore::rescore_with_percolator Starting percolator with command percolator --weights /home/karim/projects/oktoberfest/tutorials/out/percolator/rescore.percolator.weights.csv                     --num-threads 3                     --subset-max-train 500000                     --post-processing-tdc                     --testFDR 0.01                     --trainFDR 0.01                     --results-psms /home/karim/projects/oktoberfest/tutorials/out/percolator/rescore.percolator.psms.txt                     --decoy-results-psms /home/karim/projects/oktoberfest/tutorials/out/percolator/rescore.percolator.decoy.psms.txt                     --results-peptides /home/karim/projects/oktoberfest/tutorials/out/percolator/rescore.percolator.peptides.txt                     --decoy-results-peptides /home/karim/projects/oktoberfest/tutorials/out/percolator/rescore.percolator.decoy.peptides.txt                     /home/karim/projects/oktoberfest

2023-11-07 16:34:01,441 - INFO - oktoberfest.rescore.rescore::rescore_with_percolator Finished rescoring using percolator.


In [41]:
ok.re.rescore_with_percolator(original_features, original_percolator)

2023-11-07 16:34:01,516 - INFO - oktoberfest.rescore.rescore::rescore_with_percolator Starting percolator with command percolator --weights /home/karim/projects/oktoberfest/tutorials/out/percolator/original.percolator.weights.csv                     --num-threads 3                     --subset-max-train 500000                     --post-processing-tdc                     --testFDR 0.01                     --trainFDR 0.01                     --results-psms /home/karim/projects/oktoberfest/tutorials/out/percolator/original.percolator.psms.txt                     --decoy-results-psms /home/karim/projects/oktoberfest/tutorials/out/percolator/original.percolator.decoy.psms.txt                     --results-peptides /home/karim/projects/oktoberfest/tutorials/out/percolator/original.percolator.peptides.txt                     --decoy-results-peptides /home/karim/projects/oktoberfest/tutorials/out/percolator/original.percolator.decoy.peptides.txt                     /home/karim/projects/oktobe

# Visualisations

In [42]:
# Plots path 
plots = '/home/karim/projects/oktoberfest/tutorials/out/plots'

# Generate histogram of the score distribution for targets and decoys.
from pathlib import Path
perc_output_path = Path('/home/karim/projects/oktoberfest/tutorials/out/percolator')
peptide_decoy_df = pd.read_csv(perc_output_path / 'rescore.percolator.decoy.peptides.txt', sep='\t')

peptide_target_df = pd.read_csv(perc_output_path / 'rescore.percolator.peptides.txt', sep='\t')

display(peptide_decoy_df)

ok.pl.plot_score_distribution(peptide_target_df, peptide_decoy_df, level = 'peptide', filename=plots)

,PSMId,filename,score,q-value,posterior_error_prob,peptide,proteinIds
0,GN20170722_SK_HLA_G0103_R2_02-7450-KINHESLKI-3-4,GN20170722_SK_HLA_G0103_R2_02,0.201566,0.002401,0.039482,_.KINHESLKI._,KINHESLKI
1,GN20170722_SK_HLA_G0103_R1_01-15631-QLLEKRNIL-2-9,GN20170722_SK_HLA_G0103_R1_01,0.160470,0.003021,0.058471,_.QLLEKRNIL._,QLLEKRNIL
2,GN20170722_SK_HLA_G0103_R1_01-5581-IRNDHIRL-3-4,GN20170722_SK_HLA_G0103_R1_01,0.142857,0.003813,0.069125,_.IRNDHIRL._,IRNDHIRL
3,GN20170722_SK_HLA_G0103_R1_01-6469-LKSRKALDL-3-5,GN20170722_SK_HLA_G0103_R1_01,0.136986,0.004198,0.073076,_.LKSRKALDL._,LKSRKALDL
4,GN20170722_SK_HLA_G0103_R1_01-21958-PPPPPPPPPP...,GN20170722_SK_HLA_G0103_R1_01,0.120352,0.004680,0.085481,_.PPPPPPPPPPPPPPPPPPPPQAEL._,PPPPPPPPPPPPPPPPPPPPQAEL
...,...,...,...,...,...,...,...
11684,GN20170722_SK_HLA_G0103_R2_01-46428-MKSKKKKKKL...,GN20170722_SK_HLA_G0103_R2_01,-3.435420,0.879004,1.000000,_.MKSKKKKKKL._,MKSKKKKKKL
11685,GN20170722_SK_HLA_G0103_R1_02-52044-LEDSGDEC[U...,GN20170722_SK_HLA_G0103_R1_02,-3.444230,0.879004,1.000000,_.LEDSGDEC[UNIMOD:4]DE._,LEDSGDEC[UNIMOD:4]DE
11686,GN20170722_SK_HLA_G0103_R2_01-46489-VVKKKKKKE-2-6,GN20170722_SK_HLA_G0103_R2_01,-3.524460,0.879004,1.000000,_.VVKKKKKKE._,VVKKKKKKE
11687,GN20170722_SK_HLA_G0103_R2_01-51792-ELIRIRRRIK...,GN20170722_SK_HLA_G0103_R2_01,-3.650680,0.879004,1.000000,_.ELIRIRRRIKVK._,ELIRIRRRIKVK


In [44]:
ok.pl.plot_all(perc_output_path)